In [2]:
from sklearn.feature_extraction.text import CountVectorizer

from tqdm import tqdm
import torch
import pandas as pd
import joblib
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score
from matplotlib import pyplot as plt
import numpy as np
import os
from core import utils
from core import model_utils, data_utils, constants
from sklearn.model_selection import train_test_split
from core.attack_utils import mimicry
from core import attack_utils
from datetime import datetime

from core.attack_utils import mimicry

#Calculate AUT base on f1 list
def aut_score(f1_list):
    aut = 0
    for i in range(len(f1_list)-1):
        aut += (f1_list[i]+f1_list[i+1])/2
    aut = aut/(len(f1_list)-1)
    print(aut)
    return aut

#Evaluate AUT
def evaluate_aut(model,x_test,y_test,year='2015'):
    samples_info = pd.read_json(os.path.join(constants.DREBIN_DATA_DIR,"extended-features-meta.json"))
    timelines = np.array([
                 '2015-01','2015-02','2015-03','2015-04','2015-05','2015-06',
                 '2015-07','2015-08','2015-09','2015-10','2015-11','2015-12',
                 '2016-01','2016-02','2016-03','2016-04','2016-05','2016-06',
                 '2016-07','2016-08','2016-09','2016-10','2016-11','2016-12',
                 '2017-01','2017-02','2017-03','2017-04','2017-05','2017-06',
                 '2017-07','2017-08','2017-09','2017-10','2017-11','2017-12',
                 '2018-01','2018-02','2018-03','2018-04','2018-05','2018-06',
                 '2018-07','2018-08','2018-09','2018-10','2018-11','2018-12',
                 '2019-01'])
    test_samples = samples_info[(samples_info.dex_date>year)]
    timelines = timelines[timelines>year]
    f1_list = []
    goodware_count = []
    malware_count = []
    for i in range(len(timelines)-1):
        indicies = (test_samples.dex_date>timelines[i])&(test_samples.dex_date<timelines[i+1])
        x_t,y_t = x_test[indicies],y_test[indicies]
        if isinstance(model,LinearSVC):
            r = model.predict(x_t)
        else:
            r = model.predict(x_t)>0.5
        f1_list.append(f1_score(y_t,r))
        goodware_count.append(y_t.shape[0]-y_t.sum())
        malware_count.append(y_t.sum())
    return f1_list, aut_score(f1_list)

def evaluate_aut_r(r,y_test,year='2015'):
    samples_info = pd.read_json(os.path.join(constants.DREBIN_DATA_DIR,"extended-features-meta.json"))
    timelines = np.array([
                 '2015-01','2015-02','2015-03','2015-04','2015-05','2015-06',
                 '2015-07','2015-08','2015-09','2015-10','2015-11','2015-12',
                 '2016-01','2016-02','2016-03','2016-04','2016-05','2016-06',
                 '2016-07','2016-08','2016-09','2016-10','2016-11','2016-12',
                 '2017-01','2017-02','2017-03','2017-04','2017-05','2017-06',
                 '2017-07','2017-08','2017-09','2017-10','2017-11','2017-12',
                 '2018-01','2018-02','2018-03','2018-04','2018-05','2018-06',
                 '2018-07','2018-08','2018-09','2018-10','2018-11','2018-12',
                 '2019-01'])
    test_samples = samples_info[(samples_info.dex_date>year)]
    timelines = timelines[timelines>year]
    f1_list = []
    goodware_count = []
    malware_count = []
    for i in range(len(timelines)-1):
        indicies = (test_samples.dex_date>timelines[i])&(test_samples.dex_date<timelines[i+1])
        r_t,y_t = r[indicies],y_test[indicies]
        f1_list.append(f1_score(y_t,r_t))
        #print(f1_list[-1])
        goodware_count.append(y_t.shape[0]-y_t.sum())
        malware_count.append(y_t.sum())
        #print(goodware_count[-1],malware_count[-1])
    return f1_list, aut_score(f1_list)

## load dataset

In [ ]:
samples_info = pd.read_json("../transcend/extended-features-meta.json")
x_train_,y_train_,x_test,y_test = data_utils.load_drebin_dataset('2015',False)
features, feature_names, name_feat, feat_name = data_utils.load_features([],'drebin','2015',False)
x_train,x_val,y_train,y_val = train_test_split(x_train_,y_train_,test_size=0.05,random_state = 3)

## Mark which features are modifiable

In [ ]:
#note api features
api_idx = [i for i in range(len(feature_names)) if 'calls' in feature_names[i].split('::')[0]]
removable_fs = [i for i in range(len(feature_names)) if 'api_calls' in feature_names[i].split('::')[0]
               or 'activities' in feature_names[i].split('::')[0] 
                or 's_and_r' in feature_names[i].split('::')[0]
               or 'providers' in feature_names[i].split('::')[0]]
manipulation = np.zeros(x_train_.shape[1])
manipulation[removable_fs] = 1
apis = np.zeros(x_train_.shape[1])
apis[api_idx] = 1
joblib.dump([apis,manipulation],"materials/feat_info.pkl")

### SVM

In [ ]:
if not os.path.exists('models/drebin/linearsvm_drebin_full.pkl'):
    basesvm = model_utils.load_model('linearsvm','drebin','models/drebin/','linearsvm_drebin_full')
else:
    basesvm = model_utils.train_model('linearsvm','drebin',x_train,y_train,x_test,y_test)
    model_utils.save_model('linearsvm',basesvm,'models/drebin/','linearsvm_drebin_full')
r = basesvm.predict(x_test)
print(classification_report(r,y_test,digits=5))
f1_list, aut = evaluate_aut(basesvm,x_test,y_test,year='2015')
print(aut)

#### mimicry attacks on SVM

In [ ]:
r = basesvm.predict(x_train)
indicies = np.where((r!=0)&(y_train!=0))[0][:1000]
ben_x = x_train[np.where((r==0)&(y_train==0))]

success_flag,x_mod = mimicry(basesvm, x_train[indicies][:500], ben_x, torch.Tensor(manipulation), None, 1)
print((success_flag==1).sum()/len(success_flag))
success_flag,x_mod = mimicry(basesvm, x_train[indicies][:500], ben_x, torch.Tensor(manipulation), None, 10)
print((success_flag==1).sum()/len(success_flag))
success_flag,x_mod = mimicry(basesvm, x_train[indicies][:500], ben_x, torch.Tensor(manipulation), None, 30)
print((success_flag==1).sum()/len(success_flag))

#### Draw the distribution of features

In [ ]:
n_samples = np.array(x_train_[:,idx].sum(axis=0))[0,:]
sorted_weight = abs(basesvm.coef_[0]).argsort()
x= x_train_[:,sorted_weight[-1000:]]

l = [0,1,2,5,10,50,100,10000000]
lst = []
for i in range(len(l)-1):
    lst.append(x[(x.sum(axis=0)>l[i])&(x.sum(axis=0)<=l[i+1])].shape[1])

# 假设你有一些数据
y = lst
label = ['1','2','5','10','50','100','>100']
plt.rcParams['font.sans-serif'] = ['Times New Roman']
x = np.arange(len(y))

fig, ax = plt.subplots()
plt.style.use('bmh')
plt.grid(axis='x')

# 设置背景透明度
ax.patch.set_alpha(0)

plt.xlabel('Occurrences', fontsize=14)
plt.ylabel('Feature counts', fontsize=14)
ax.bar(x, y, hatch='//')
plt.xticks(x, label,size=14,rotation=0)
plt.yticks(size=14)
#plt.ylim(330000,430000)
fig.subplots_adjust(bottom=0.5)
# 设置坐标轴的背景透明度和颜色
ax.xaxis.set_tick_params(color='black', labelcolor='black', pad=10)
#
plt.savefig("images/drebin_features.png",bbox_inches='tight')
plt.savefig("images/drebin_features.pdf",bbox_inches='tight')
plt.show()

## NN-Selected

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
selector = SelectFromModel(basesvm, prefit=True, max_features=1000)
idx = np.where(selector.get_support()==True)[0]

x_train_selected = np.array(x_train[:,idx].todense())
x_test_selected = np.array(x_test[:,idx].todense())
x_val_selected = np.array(x_val[:,idx].todense())

if os.path.exists('models/drebin/nn_drebin_selected.pkl'):
    nn_selected = model_utils.load_model('nn','drebin','models/drebin/','nn_drebin_selected',x_train_selected.shape[1])
else:
    nn_selected = model_utils.train_model('nn','drebin',x_train_selected,y_train,x_val_selected,y_val,300,'')
    model_utils.save_model('nn',nn_selected,'models/drebin/','nn_drebin_selected')
r=nn_selected.predict(x_test_selected)>0.5
print(classification_report(r,y_test,digits=5))
f1_list_selected, aut_selected = evaluate_aut(nn_selected,x_test_selected,y_test,year='2015')

#### Saving modifiable features for PAD and backdoor attack

In [ ]:
joblib.dump([x_train_selected, y_train, x_val_selected, y_val, x_test_selected, y_test],'materials/drebin_selected.pkl')
features, feature_names, name_feat, feat_name = data_utils.load_features([],'drebin','2015',False,idx)

api_idx = [i for i in range(len(feature_names)) if 'calls' in feature_names[i].split('::')[0]]
removable_fs = [i for i in range(len(feature_names)) if 'api_calls' in feature_names[i].split('::')[0]
               or 'activities' in feature_names[i].split('::')[0] 
                or 's_and_r' in feature_names[i].split('::')[0]
               or 'providers' in feature_names[i].split('::')[0]]
manipulation = np.zeros(1000)
manipulation[removable_fs] = 1
apis = np.zeros(1000)
apis[api_idx] = 1
joblib.dump([apis,manipulation],"materials/feat_info_selected.pkl")

#### Mimicry on NN-selected

In [ ]:
#print(f1_list_selected,aut_selected)##44451
indicies = np.where((r!=0)&(y_test!=0))[0][:1000]
r_train = nn_selected.predict(x_train_selected)>0.5
ben_x = x_train_selected[np.where((r_train==0)&(y_train==0))]

success_flag,x_mod = mimicry(nn_selected, x_test_selected[indicies], ben_x, torch.Tensor(manipulation)[idx], None, 1)
print((success_flag==0).sum()/len(success_flag))
success_flag,x_mod = mimicry(nn_selected, x_test_selected[indicies], ben_x, torch.Tensor(manipulation)[idx], None, 10)
print((success_flag==0).sum()/len(success_flag))
success_flag,x_mod = mimicry(nn_selected, x_test_selected[indicies], ben_x, torch.Tensor(manipulation)[idx], None, 30)
print((success_flag==0).sum()/len(success_flag))

## NN-Dense

In [ ]:
feature_counts = np.array(x_train_.sum(axis=0))[0]
dense_feature = feature_counts.argsort()[-682:] #Features with at least 1% density
x_train_d = np.array(x_train[:,dense_feature].todense())
x_test_d = np.array(x_test[:,dense_feature].todense())
x_val_d = np.array(x_val[:,dense_feature].todense())
joblib.dump([x_train_d,x_val_d,x_test_d,y_train,y_val,y_test],'materials/drebin_2015_682.pkl')
#x_train_d,x_val_d,x_test_d,y_train,y_val,y_test = joblib.load('materials/drebin_2015_682.pkl')

In [ ]:
if os.path.exists('models/drebin/nn_drebin_d.pkl'):
    nn_d = model_utils.load_model('nn','drebin','models/drebin/','nn_drebin_d',x_train_d.shape[1])
else:
    nn_d = model_utils.train_model('nn','drebin',x_train_d,y_train,x_test_d,y_test,300,'')
    model_utils.save_model('nn',nn_d,'models/drebin/','nn_drebin_d')
r=nn_d.predict(x_test_d)>0.5
print(classification_report(r,y_test,digits=5))
f1_list_d, aut_d = evaluate_aut(nn_d,x_test_d,y_test,year='2015')

#### Saving modifiable features for PAD and backdoor attack

In [ ]:
features, feature_names, name_feat, feat_name = data_utils.load_features([],'drebin','2015',False,dense_feature)
api_idx = [i for i in range(len(feature_names)) if 'calls' in feature_names[i].split('::')[0]]
removable_fs = [i for i in range(len(feature_names)) if 'api_calls' in feature_names[i].split('::')[0]
               or 'activities' in feature_names[i].split('::')[0] 
                or 's_and_r' in feature_names[i].split('::')[0]
               or 'providers' in feature_names[i].split('::')[0]]
manipulation = np.zeros(682)
manipulation[removable_fs] = 1
apis = np.zeros(682)
apis[api_idx] = 1
joblib.dump([apis,manipulation],"materials/feat_info_dense.pkl")

#### Mimicry on NN-dense

In [ ]:
idx = dense_feature
indicies = np.where((r!=0)&(y_test!=0))[0][:1000]
r_train = nn_d.predict(x_train_d)>0.5
ben_x = x_train_d[np.where((r_train==0)&(y_train==0))]

success_flag,x_mod = mimicry(nn_d, x_test_d[indicies], ben_x, torch.Tensor(manipulation)[idx], None, 1)
print((success_flag==0).sum()/len(success_flag))
success_flag,x_mod = mimicry(nn_d, x_test_d[indicies], ben_x, torch.Tensor(manipulation)[idx], None, 10)
print((success_flag==0).sum()/len(success_flag))
success_flag,x_mod = mimicry(nn_d, x_test_d[indicies], ben_x, torch.Tensor(manipulation)[idx], None, 30)
print((success_flag==0).sum()/len(success_flag))

## NN-Bundle

In [ ]:
x_train_bundle_,x_test_bundle,y_train_bundle_,y_test,processor = data_utils.load_compressed_drebin(2015,4)
x_train_bundle,x_val_bundle,y_train,y_val = train_test_split(x_train_bundle_,y_train_bundle_,test_size=0.05,random_state = 3)
if os.path.exists('models/drebin/nn_drebin_bundle.pkl'):
    nn_bundle = model_utils.load_model('nn','drebin','models/drebin/','nn_drebin_bundle',x_train_bundle.shape[1])
else:
    nn_bundle = model_utils.train_model('nn','drebin',x_train_bundle,y_train,x_val_bundle,y_val,300,'')
    model_utils.save_model('nn',nn_bundle,'models/drebin/','nn_drebin_bundle')
r=nn_bundle.predict(x_test_bundle)>0.5
print(classification_report(r,y_test,digits=5))
f1_list_bundle, aut_bundle = evaluate_aut(nn_bundle,x_test_bundle,y_test,year='2015')

#### Mimicry on NN-bundle

In [ ]:
print(f1_list_bundle, aut_bundle)
r=nn_bundle.predict(x_train_bundle)>0.5
indicies = np.where((r!=0)&(y_train!=0))[0][:1000]
ben_x = x_train[np.where((r==0)&(y_train==0))]

apis,manipulation = joblib.load("materials/feat_info.pkl")

success_flag,x_mod = mimicry(nn_bundle, x_train[indicies][:500], ben_x, torch.Tensor(manipulation), processor, 1)
print((success_flag==1).sum()/len(success_flag))
success_flag,x_mod = mimicry(nn_bundle, x_train[indicies][:500], ben_x, torch.Tensor(manipulation), processor, 10)
print((success_flag==1).sum()/len(success_flag))
success_flag,x_mod = mimicry(nn_bundle, x_train[indicies][:500], ben_x, torch.Tensor(manipulation), processor, 30)
print((success_flag==1).sum()/len(success_flag))

### NN-Density

In [ ]:
method = 'density0'
if os.path.exists(f'models/drebin/nn_drebin_{method}.pkl'):
    nn_density = model_utils.load_model('nn','drebin','models/drebin/',f'nn_drebin_{method}',x_train_bundle.shape[1])
else:
    nn_density = model_utils.train_model('nn','drebin',x_train_bundle,y_train,x_val_bundle,y_val,300,f'{method}')
    model_utils.save_model('nn',nn_density,'models/drebin/',f'nn_drebin_{method}')
r=nn_density.predict(x_test_bundle)>0.5
print(classification_report(r,y_test,digits=5))
f1_list_density, aut_density= evaluate_aut(nn_density,x_test_bundle,y_test,year='2015')

In [ ]:
r=nn_density.predict(x_train_bundle)>0.5
indicies = np.where((r!=0)&(y_train!=0))[0][:1000]
ben_x = x_train[np.where((r==0)&(y_train==0))]

apis,manipulation = joblib.load("materials/feat_info.pkl")

success_flag,x_mod = mimicry(nn_density, x_train[indicies][:500], ben_x, torch.Tensor(manipulation), processor, 1)
print((success_flag==1).sum()/len(success_flag))
success_flag,x_mod = mimicry(nn_density, x_train[indicies][:500], ben_x, torch.Tensor(manipulation), processor, 10)
print((success_flag==1).sum()/len(success_flag))
success_flag,x_mod = mimicry(nn_density, x_train[indicies][:500], ben_x, torch.Tensor(manipulation), processor, 30)
print((success_flag==1).sum()/len(success_flag))

### Draw the lines

In [ ]:
f1_list, aut = [0.7625899280575541, 0.7520661157024793, 0.7312348668280872, 0.6946107784431138, 0.7667560321715818, 0.6337209302325582, 0.6973684210526315, 0.7014613778705637, 0.7166123778501629, 0.6717850287907869, 0.5922165820642977, 0.5321100917431192, 0.4590818363273452, 0.4797601199400299, 0.4585152838427948, 0.41696113074204944, 0.49695121951219506, 0.5104408352668214, 0.5619223659889093, 0.508833922261484, 0.5663716814159292, 0.5116279069767441, 0.3225806451612903, 0.4, 0.2408111533586819, 0.295, 0.24142312579415504, 0.3030998851894374, 0.24009603841536614, 0.372960372960373, 0.4823906083244397, 0.421978021978022, 0.41683366733466937, 0.46105919003115264, 0.4206008583690987, 0.4237837837837838, 0.4362549800796813, 0.4041884816753927, 0.2968845448992059, 0.38078291814946613, 0.3491755577109602, 0.503157894736842, 0.4684491978609625, 0.48297872340425535, 0.5166130760986066, 0.19546742209631726, 0.10829103214890018, 0.047058823529411764], 0.46915094596549417
f1_list_selected,aut_selected = [0.7626459143968871, 0.7074235807860262, 0.6975609756097562, 0.6162790697674418, 0.6441558441558441, 0.562189054726368, 0.6303501945525292, 0.5996343692870201, 0.7419354838709677, 0.696461824953445, 0.5649913344887348, 0.48648648648648646, 0.4665391969407266, 0.5270457697642164, 0.4, 0.4260869565217391, 0.5620736698499318, 0.4943310657596373, 0.5916955017301038, 0.5219399538106235, 0.4545454545454545, 0.5891472868217054, 0.26666666666666666, 0.2222222222222222, 0.3047375160051216, 0.3341708542713568, 0.2912621359223301, 0.3352872215709261, 0.2518518518518519, 0.3990208078335373, 0.501138952164009, 0.44815256257449343, 0.47513812154696133, 0.5197860962566845, 0.45999999999999996, 0.42224744608399545, 0.4131147540983606, 0.4018161180476731, 0.29820051413881743, 0.3988657844990548, 0.38469493278179945, 0.5329087048832271, 0.506050605060506, 0.5617021276595744, 0.603537981269511, 0.25815602836879437, 0.1861252115059222, 0.08], 0.47189470742362977
f1_list_d, aut_d = [0.8074074074074075, 0.6923076923076923, 0.6962962962962963, 0.6118980169971672, 0.6582914572864321, 0.5440806045340051, 0.62109375, 0.6338028169014085, 0.6656101426307448, 0.6990990990990991, 0.6466876971608833, 0.5894039735099338, 0.5225563909774437, 0.5072046109510087, 0.5141065830721003, 0.5372168284789643, 0.5563480741797432, 0.6299559471365639, 0.6330434782608696, 0.6596858638743456, 0.6506024096385542, 0.5238095238095238, 0.47058823529411764, 0.2222222222222222, 0.3424317617866005, 0.3337515683814304, 0.31013431013431014, 0.4078341013824885, 0.35990338164251207, 0.4258373205741627, 0.48914285714285716, 0.47206385404789053, 0.4718853362734289, 0.505795574288725, 0.4924406047516199, 0.45336225596529284, 0.43373493975903615, 0.31490384615384615, 0.37446808510638296, 0.4889705882352941, 0.4649805447470817, 0.6456852791878173, 0.6215384615384615, 0.6484848484848484, 0.6613065326633166, 0.35135135135135137, 0.25, 0.18181818181818182], 0.5170325938900143
f1_list_bundle, aut_bundle = [0.7928571428571428, 0.7160493827160493, 0.6894865525672371, 0.7024128686327078, 0.6649874055415617, 0.5357142857142857, 0.6666666666666666, 0.6420664206642066, 0.6708268330733229, 0.6546052631578947, 0.617816091954023, 0.5978090766823161, 0.6256239600665557, 0.5541838134430727, 0.6218009478672986, 0.5254413291796469, 0.539454806312769, 0.5528089887640449, 0.6068222621184919, 0.6885919835560124, 0.6666666666666666, 0.4915254237288136, 0.3333333333333333, 0.4, 0.33985330073349634, 0.3130434782608696, 0.2911392405063291, 0.4165733482642777, 0.2847301951779564, 0.4004424778761062, 0.48497854077253216, 0.4323189926547744, 0.5118679050567595, 0.48502994011976047, 0.514456630109671, 0.47731755424063116, 0.4854368932038835, 0.4133738601823708, 0.3561470215462611, 0.5095119933829612, 0.4949771689497717, 0.6199324324324325, 0.631762652705061, 0.6655290102389079, 0.6878761822871883, 0.31422018348623854, 0.2136986301369863, 0.16666666666666666], 0.5231632744573215
f1_list_density, aut_density = [0.8489208633093526, 0.7654320987654321, 0.7215496368038741, 0.6611111111111111, 0.7619047619047619, 0.6069651741293532, 0.6856060606060606, 0.691358024691358, 0.7689873417721519, 0.8, 0.7712609970674487, 0.7538461538461538, 0.762987012987013, 0.7005208333333334, 0.7078972407231209, 0.6486486486486487, 0.6535764375876578, 0.6203904555314533, 0.6382252559726962, 0.7492447129909365, 0.7251908396946565, 0.5950413223140496, 0.41379310344827586, 0.4, 0.4336175395858709, 0.4224343675417661, 0.40441176470588236, 0.47520184544405997, 0.41092636579572445, 0.4960362400906002, 0.509009009009009, 0.4759725400457666, 0.4849162011173184, 0.5238095238095238, 0.5231431646932185, 0.488272921108742, 0.49060773480662984, 0.3104693140794224, 0.40460763138948885, 0.5596244131455399, 0.5383141762452107, 0.6965648854961832, 0.6467757459095284, 0.720532319391635, 0.7423076923076923, 0.34850455136540964, 0.19829059829059828, 0.1794871794871795], 0.5834487615043112

plt.style.use('bmh')
name = "images/drebin_features"
fig = plt.figure(figsize=(6,4)) #创建绘图对象
ax = fig.add_subplot(111)
plt.grid(axis='x')
ax.patch.set_alpha(0)
series = range(1,48)
ax.set_xlabel("Timeline",fontsize=15)
ax.set_ylabel('F1 Score',fontsize=15)
line1 = ax.plot(range(1,len(f1_list)+1),f1_list,linestyle='-.',alpha=0.5,label=f"SVM (AUT:{aut:.5f})")[0]
line2 = ax.plot(range(1,len(f1_list)+1),f1_list_selected,linestyle='-.',alpha=0.5,label=f"NN-Selected (AUT:{aut_selected:.5f})")[0]
line3 = ax.plot(range(1,len(f1_list)+1),f1_list_d,linestyle='-.',alpha=0.5,label=f"NN-Density (AUT:{aut_d:.5f})")[0]
line4 = ax.plot(range(1,len(f1_list)+1),f1_list_bundle,linestyle='-.',alpha=0.5,label=f"NN-Bundle (AUT:{aut_bundle:.5f})")[0]
line5 = ax.plot(range(1,len(f1_list)+1),f1_list_density,linestyle='-',alpha=1,label=f"NN-DB (AUT:{aut_density:.5f})")[0]

second_legend_handles = [line1, line5,line2,line3, line4]
second_legend_labels = [line1.get_label(), line5.get_label(), line2.get_label(), line3.get_label(), line4.get_label()]
ax.legend(handles=second_legend_handles, labels=second_legend_labels, loc='lower left', fontsize=11, facecolor="none", frameon=False)

x_ticks = np.arange(1, len(f1_list), 3)  # 设置要显示的刻度位置
plt.xticks(x_ticks,x_ticks, fontsize=13,rotation=30)
plt.yticks(np.arange(0,0.9,0.1),fontsize=13)
plt.subplots_adjust(wspace=0.1)

plt.savefig(name+".pdf", bbox_inches='tight') #保存图
plt.savefig(name, bbox_inches='tight') #保存图
plt.show()  #显示图

## PAD

In [ ]:
from pad.core.defense import AMalwareDetectionPAD
from pad.core.defense import AdvMalwareDetectorICNN
from pad.core.defense import MalwareDetectionDNN

name = '20240605-113157'#'basepad'
#name = '20240623-231424'#'basepad+db+pad with restricted features 
#name = '20240623-231404'#'basepad+pad with restricted features 
args = {'dense_hidden_units':[1024,512,256],
        'dropout':0.6,
        'alpha_':0.2,
        'smooth':False,
        'proc_number':10,
       }
model = MalwareDetectionDNN(558,
                            2,
                            device='cpu',
                            name=name,
                            **args
                            )
model = AdvMalwareDetectorICNN(model,
                            input_size=558,
                            n_classes=2,
                            device='cpu',
                            name=name,
                            **args
                            )
max_adv_training_model = AMalwareDetectionPAD(model, None, None)
max_adv_training_model.load()
print(f'Load {name} pad')

#evaluate performance
y_cent, y_prob, y_true = model.inference(utils.data_iter(256,x_test_bundle, y_test, False))
r = y_cent.argmax(axis=1)
f1_list_pad, aut_pad= evaluate_aut(r.numpy(),y_test,year='2015')
print(classification_report(r,y_test,digits=5))

#evaluate mimicry
y_cent, y_prob, y_true = model.inference(utils.data_iter(256,x_train_bundle, y_train, False))
r = y_cent.argmax(axis=1)
indicies = np.where((r!=0)&(y_train!=0))[0][:1000] #get 1000 malwares in training set
ben_x = x_train[np.where((r==0)&(y_train==0))] #get goodwares
print(classification_report(r,y_train,digits=5))

success_flag,x_mod = mimicry(model, x_train[indicies][:500], ben_x, torch.Tensor(manipulation), processor, 1)
print((success_flag==1).sum()/len(success_flag))
success_flag,x_mod = mimicry(model, x_train[indicies][:500], ben_x, torch.Tensor(manipulation), processor, 10)
print((success_flag==1).sum()/len(success_flag))
success_flag,x_mod = mimicry(model, x_train[indicies][:500], ben_x, torch.Tensor(manipulation), processor, 30)
print((success_flag==1).sum()/len(success_flag))

In [ ]:
#evaluate backdoors
f_s,v_s=joblib.load('materials/trigger_VR_drebin_nn_drebin_bundle_all.pkl')
fn=16
idx = np.where(np.array(v_s)!=0)[0]
if len(idx)<fn:
    v_s = [1]*len(f_s)
    idx = np.where(np.array(v_s)!=0)[0]
f_s = np.array(f_s)[idx].tolist()[:fn]
v_s = np.array(v_s)[idx].tolist()[:fn]

x_t = x_test_bundle[y_test==1].copy()
y_cent, y_prob, y_true = model.inference(utils.data_iter(256,x_t, np.array([1]*x_t.shape[0]), False))
r = y_cent.argmax(axis=1)
print(r[r==y_true].shape[0]/r.shape[0])#performance before backdoor

#inject backdoor
for i,j in list(zip(f_s,v_s))[:16]:
    x_t[:,i] = j
    
y_cent, y_prob, y_true = model.inference(utils.data_iter(256,x_t, np.array([1]*x_t.shape[0]), False))
r = y_cent.argmax(axis=1)
print(r[r==y_true].shape[0]/r.shape[0])#performance after backdoor

### Evaluate PAD on NN-selected

In [ ]:
name = '20240703-194850'
args = {'dense_hidden_units':[1024,512,256],
        'dropout':0.6,
        'alpha_':0.2,
        'smooth':False,
        'proc_number':10,
       }
model = MalwareDetectionDNN(1000,
                            2,
                            device='cpu',
                            name=name,
                            **args
                            )
model = AdvMalwareDetectorICNN(model,
                            input_size=1000,
                            n_classes=2,
                            device='cpu',
                            name=name,
                            **args
                            )
max_adv_training_model = AMalwareDetectionPAD(model, None, None)
max_adv_training_model.load()
print(f'Load {name} pad')

y_cent, y_prob, y_true = model.inference(utils.data_iter(256,x_test_selected, y_test, False))
r = y_cent.argmax(axis=1)
f1_list_pad, aut_pad= evaluate_aut_r(r.numpy(),y_test,year='2015')#AUT: 50.3%

y_cent, y_prob, y_true = model.inference(utils.data_iter(256,x_train_selected, y_train, False))
r = y_cent.argmax(axis=1)
indicies = np.where((r!=0)&(y_true!=0))[0][:1000]
#r_test = nn_bundle.predict(x_test_bundle)>0.5
ben_x = x_train_selected[np.where((r==0)&(y_true==0))]

success_flag,x_mod = mimicry(model, x_train_selected[indicies][:500], ben_x, torch.Tensor(manipulation[idx]), None, 1, False)
print((success_flag==1).sum()/len(success_flag))
success_flag,x_mod = mimicry(model, x_train_selected[indicies][:500], ben_x, torch.Tensor(manipulation[idx]), None, 10, False)
print((success_flag==1).sum()/len(success_flag))
success_flag,x_mod = mimicry(model, x_train_selected[indicies][:500], ben_x, torch.Tensor(manipulation[idx]), None, 30, False)
print((success_flag==1).sum()/len(success_flag))

f_s,v_s=joblib.load('materials/trigger_VR_drebin_nn_drebin_selected_all.pkl')
x_t = x_test_selected[y_test==1].copy()
y_cent, y_prob, y_true = model.inference(utils.data_iter(256,x_t, np.array([1]*x_t.shape[0]), False))
r = y_cent.argmax(axis=1)

fn=16
idx = np.where(np.array(v_s)!=0)[0]
if len(idx)<fn:
    v_s = [1]*len(f_s)
    idx = np.where(np.array(v_s)!=0)[0]
f_s = np.array(f_s)[idx].tolist()[:fn]
v_s = np.array(v_s)[idx].tolist()[:fn]

for i,j in list(zip(f_s,v_s))[:16]:
    x_t[:,i] = j

y_cent, y_prob, y_true = model.inference(utils.data_iter(256,x_t, np.array([1]*x_t.shape[0]), False))
r = y_cent.argmax(axis=1)
r[r==y_true].shape[0]/r.shape[0]

## Evaluation on APIGraph 

### Process datasets

In [442]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.svm import SVC
from scipy.sparse import vstack
vec = DictVectorizer()
X = vec.fit_transform(X)
y = np.asarray(y)
# Partition dataset via TESSERACT
splits = temporal.time_aware_train_test_split(X, y, t, train_size=12, test_size=1, granularity='month')

x_train_, x_test, y_train_, y_test, t_train, t_test = splits
x_test = vstack(x_test)
y_test = np.hstack(y_test)

feature_names = vec.get_feature_names_out()

#Download the APIGrap first
with open('APIGraph/src/res/method_cluster_mapping_%d.pkl' % (2000), 'rb') as f:
    method_cluster_mapping = pickle.load(f) 
#method_cluster_mapping = {key.lower(): value for key, value in method_cluster_mapping.items()}

api_idx = [i for i in range(len(feature_names)) if 'calls' in feature_names[i].split('::')[0]]
api_features = [feature_names[i] for i in api_idx]
x_apis = x_train_[:,api_idx].todense()
x_apis_test = x_test[:,api_idx].todense()

#Some APIs are not in current list, we do a simple procession for them
processed_apis = []
count = 0
count1 = 0
for idx,api in enumerate(api_features):
    api_type = api.split('::')[0]
    api = api.split('::')[1]
    api = api.replace('/','.').replace(';->','.')
    if 'Cipher' in api:#all cipher are compressed into 1 feature
        processed_apis.append(1761)
    elif api in method_cluster_mapping:
        processed_apis.append(method_cluster_mapping[api])
        count1 += 1
    else:
        print(idx,api)
        while True:
            candidates = [(i,method_cluster_mapping[i],Levenshtein.ratio(api, i)) for i in method_cluster_mapping if api in i]
            if len(candidates) == 0 and '.' in api:
                api = '.'.join(api.split('.')[:-1])
                continue
            else:
                break
        candidates = sorted(candidates,key=lambda x:x[2],reverse=True)[:5]
        #candidates = [(i,method_cluster_mapping[i],Levenshtein.distance(api,idx)) for i in method_cluster_mapping if api in i]
        print(candidates)
        if candidates:
            processed_apis.append(candidates[0][1])
        else:
            processed_apis.append(api)
        count += 1

papiset = set(processed_apis)
print(len(papiset))
new_f = []
new_f_test = []
for i in papiset:
    indicies = [idx for idx,api in enumerate(processed_apis) if api == i]
    test_indicies = [idx for idx,api in enumerate(processed_apis) if api == i]
    new_f.append(np.array(x_apis[:,indicies].sum(axis=1)))
    new_f_test.append(np.array(x_apis_test[:,indicies].sum(axis=1)))
new_x = np.hstack(new_f)
new_x_test = np.hstack(new_f_test)
new_x[new_x>1] = 1
new_x_test[new_x_test>1] = 1

x_train_api,x_val_api,y_train,y_val = train_test_split(new_x,y_train_,test_size=0.05,random_state = 3)
x_test_api = new_x_test

#Only use API features to train model
if not os.path.exists('models/drebin/nn_drebin_api.pkl'):
    nn_api = model_utils.load_model('nn','drebin','models/drebin/','nn_drebin_api',x_train_api.shape[1])
else:
    nn_api = model_utils.train_model('nn','drebin',x_train_api,y_train,x_val_api,y_val,200,'density0')
    model_utils.save_model('nn',nn_api,'models/drebin/','nn_drebin_api')
r=nn_api.predict(x_test_api)>0.5
print(classification_report(r,y_test,digits=5))
f1_list_api, aut_api = evaluate_aut(nn_api,x_test_api,y_test,year='2015')
#0.38562

### APIGraph-Dense features

In [370]:
indicies = np.where(x_train_.sum(axis=0)>577)[1]
indicies = np.array(x_train_.sum(axis=0))[0].argsort()[-10000:]
indicies = [i for i in indicies if i not in api_idx]

x_train_api = np.hstack([new_x,x_train_[:,indicies].todense()])
x_test_api = np.hstack([new_x_test,x_test[:,indicies].todense()])



print(x_train_api.shape)#735
x_train_api,x_val_api,y_train,y_val = train_test_split(x_train_api,y_train_,test_size=0.05,random_state = 3)

if not os.path.exists('models/drebin/nn_drebin_api.pkl'):
    nn_api = model_utils.load_model('nn','drebin','models/drebin/','nn_drebin_api',x_train_api.shape[1])
else:
    nn_api = model_utils.train_model('nn','drebin',x_train_api,y_train,x_val_api,y_val,200,'')
    model_utils.save_model('nn',nn_api,'models/drebin/','nn_drebin_api')
r=nn_api.predict(x_test_api)>0.5
print(classification_report(r,y_test,digits=5))
f1_list_api, aut_api = evaluate_aut(nn_api,x_test_api,y_test,year='2015')

## APIGraph-Bundle

#### Save the dataset for bundling

In [ ]:
indicies = np.where(x_train_.sum(axis=0)>=50)[1]
indicies = [i for i in indicies if i not in api_idx]
x_train_api = np.hstack([new_x,x_train_[:,indicies].todense()])
x_test_api = np.hstack([new_x_test,x_test[:,indicies].todense()])
print(len(indicies))
x_train_api, x_test_api = np.array(x_train_api), np.array(x_test_api)
joblib.dump([x_train_api,y_train,x_test_api,y_test],"materials/apigraphfeatures.pkl")

In [461]:
#Load the processed dataset
x_train_api,x_test_api,y_train,y_test = joblib.load(f"materials/compressed_drebin_apigraph_4.pkl")

In [462]:
x_train_api,x_val_api,y_train,y_val = train_test_split(x_train_api,y_train_,test_size=0.05,random_state = 3)

In [ ]:
if os.path.exists('models/drebin/nn_drebin_api_bundle.pkl'):
    nn_api = model_utils.load_model('nn','drebin','models/drebin/','nn_drebin_api_bundle',x_train_api.shape[1])
else:
    nn_api = model_utils.train_model('nn','drebin',x_train_api,y_train,x_val_api,y_val,200,'')
    model_utils.save_model('nn',nn_api,'models/drebin/','nn_drebin_api_bundle')
r=nn_api.predict(x_test_api)>0.5
print(classification_report(r,y_test,digits=5))
f1_list_api, aut_api = evaluate_aut(nn_api,x_test_api,y_test,year='2015')

## APIGraph-Select

In [447]:
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
selector = SelectFromModel(basesvm, prefit=True, max_features=1000)
idx = np.where(selector.get_support()==True)[0]

In [452]:
x_train_api = np.hstack([new_x,x_train_[:,idx].todense()])
x_test_api = np.hstack([new_x_test,x_test[:,idx].todense()])

In [454]:
x_train_api,x_val_api,y_train,y_val = train_test_split(x_train_api,y_train_,test_size=0.05,random_state = 3)

In [ ]:
if not os.path.exists('models/drebin/nn_drebin_api.pkl'):
    nn_api = model_utils.load_model('nn','drebin','models/drebin/','nn_drebin_api_select',x_train_api.shape[1])
else:
    nn_api = model_utils.train_model('nn','drebin',x_train_api,y_train,x_val_api,y_val,200,'')
    model_utils.save_model('nn',nn_api,'models/drebin/','nn_drebin_api_select')
r=nn_api.predict(x_test_api)>0.5
print(classification_report(r,y_test,digits=5))
f1_list_api, aut_api = evaluate_aut(nn_api,x_test_api,y_test,year='2015')